In [1]:
import pandas as pd

In [2]:
MIN_COUNT = 25
STRIDE = 20
LENGTH = 40

In [3]:
df = pd.read_parquet(f'data/combined_articles_top{MIN_COUNT}_len{LENGTH}_stride{STRIDE}.parquet')

In [4]:
df[df.text.isna()]

,article_id,url,title,text,lower_title,count


In [5]:
import json 

buffer = [] 
BUFFER_SIZE = 500000
chunk_id = 0
articles_processed = 0

def clear_buffer(buffer):
    with open(f'data/combined_chunks_top{MIN_COUNT}_len{LENGTH}_stride{STRIDE}.jsonl', 'a', encoding='utf-8') as file:
        for row in buffer:
            file.write(json.dumps(row) + '\n')
    return []

for _, row in df.iterrows():
    if articles_processed % 10000 == 0:
        print('processed article:', articles_processed)
    start = 0
    end = LENGTH
    text = row['text'].split()
    while end < len(text):
        buffer.append({
            'article_id': row['article_id'], 
            'chunk_id': chunk_id, 
            'chunk': ' '.join(text[start : end])
        })
        chunk_id += 1
        
        start += STRIDE 
        end += STRIDE

        if len(buffer) >= BUFFER_SIZE:
            buffer = clear_buffer(buffer)
    
    if start <= len(text):
        buffer.append({
            'article_id': row['article_id'], 
            'chunk_id': chunk_id, 
            'chunk': ' '.join(text[start :])
        })
        chunk_id += 1

    articles_processed += 1

if buffer:
    buffer = clear_buffer(buffer)

processed article: 0
processed article: 10000
processed article: 20000
processed article: 30000
processed article: 40000
processed article: 50000
processed article: 60000
processed article: 70000
processed article: 80000
processed article: 90000
